In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate, ShuffleSplit

import statsmodels.api as sm
import statsmodels.formula.api as smf

%matplotlib inline
%config InlineBackend.figure_format ='retina'

sns.set_style('darkgrid')
sns.mpl.rcParams['figure.figsize'] = (15.0, 9.0)

In [2]:
df = pd.read_csv('data\kc_house_data.csv')

In [3]:
# this is all the processing and encoding of the data from the dataframe done
# in our main notebook, just condensed into one cell without visuals.

# the only processing not performed here is onehotencoding of the zipcode
# this was just done for testing, but what it illustrates is how much of an
# effect doing that had on our model.

features_to_drop = ['id']

df.date = pd.to_datetime(df.date)
features_to_drop.extend(['date'])

features_to_drop.extend(['price'])

df.at[15856,'bedrooms'] = 3

df['waterfront'].fillna('NO', inplace=True)
df['waterfront'] = df.waterfront.map({'NO': 0, 'YES': 1})

df['view'].fillna('NONE', inplace=True)
df['view'] = df.view.map(lambda x: 0 if x == 'NONE' else 1)

condition_dict = {'Very Good': 5, 'Good': 4, 'Average': 3, 'Fair': 2,
                  'Poor': 1}
df['condition'] = df.condition.map(condition_dict)

df['grade'] = df.grade.map(lambda x: x.split()[0]).astype('int')

df.sqft_basement.replace(to_replace='?', value=0.0, inplace=True)
df['sqft_basement'] = df.sqft_basement.astype('float')

df['yr_renovated'].fillna(0.0, inplace=True)

In [24]:
X = df.drop(features_to_drop, axis=1)
y = df.price

X_const = sm.add_constant(X)
model = sm.OLS(y, X_const).fit()

print('r^2 adjusted score:', model.rsquared_adj, '\n')

predictors = pd.DataFrame([])
predictors['coeff'] = model.params
predictors['pvalue'] = model.pvalues
predictors.sort_values('pvalue', ascending=True)
print('Predictors where coefficient pvalue is > 0.05:')
if len(predictors[predictors.pvalue > 0.05]) > 0:
    print(list(predictors[predictors.pvalue > 0.05].index))
else:
    print('0')
    
print('\nPredictors with Coefficient and P-value:')
display(predictors)

testmc_df = X.corr().abs().stack().reset_index().sort_values(0,ascending=False)
testmc_df['pairs'] = list(zip(testmc_df.level_0, testmc_df.level_1))
testmc_df.set_index(['pairs'], inplace=True)
testmc_df.drop(['level_0', 'level_1'], axis=1, inplace=True)
testmc_df.columns = ['mc']

print('\nCases of multicollinearity:')
if len(testmc_df[(testmc_df.mc > 0.75) & (testmc_df.mc < 1)]) > 0:
    display(testmc_df[(testmc_df.mc > 0.75) & (testmc_df.mc < 1)])
else:
    print('0')

r^2 adjusted score: 0.6985141349587404 

Predictors where coefficient pvalue is > 0.05:
0

Predictors with Coefficient and P-value:


,coeff,pvalue
const,6.407048e+06,2.925916e-02
bedrooms,-3.983197e+04,8.281094e-89
bathrooms,4.269455e+04,1.072485e-38
sqft_living,1.054736e+02,5.972471e-09
sqft_lot,1.324820e-01,5.818407e-03
floors,7.645840e+03,3.410051e-02
waterfront,6.972981e+05,0.000000e+00
view,1.153962e+05,3.138673e-107
condition,2.704103e+04,1.612298e-30
grade,9.725809e+04,0.000000e+00



Cases of multicollinearity:


,mc
pairs,
"(sqft_living, sqft_above)",0.876448
"(sqft_above, sqft_living)",0.876448
"(grade, sqft_living)",0.762779
"(sqft_living, grade)",0.762779
"(sqft_living, sqft_living15)",0.756402
"(sqft_living15, sqft_living)",0.756402
"(grade, sqft_above)",0.756073
"(sqft_above, grade)",0.756073
"(bathrooms, sqft_living)",0.755758


In [29]:
def model_evaluation(df, target, features_to_drop):
    """
    This helper function sets up a linear regression model, evaluates the r^2
    score and reports on the features' coefficients and p-values, and any
    recommendations for removal based on p-value or multicollinearity.
    
    It takes in the dataframe, what the dependent variable is (target) and if
    there are features to drop from the dataframe for the model.
    """
    
    X = df.drop(features_to_drop, axis=1)
    y = df[target]

    X_const = sm.add_constant(X)
    model = sm.OLS(y, X_const).fit()

    print('r^2 adjusted score:', model.rsquared_adj, '\n')

    predictors = pd.DataFrame([])
    predictors['coeff'] = model.params
    predictors['pvalue'] = model.pvalues
    predictors.sort_values('pvalue', ascending=True)
    print('Predictors where coefficient pvalue is > 0.05:')
    if len(predictors[predictors.pvalue > 0.05]) > 0:
        print(list(predictors[predictors.pvalue > 0.05].index))
    else:
        print('0')

    print('\nPredictors with Coefficient and P-value:')
    display(predictors)

    testmc_df = X.corr().abs().stack().reset_index().sort_values(0,ascending=False)
    testmc_df['pairs'] = list(zip(testmc_df.level_0, testmc_df.level_1))
    testmc_df.set_index(['pairs'], inplace=True)
    testmc_df.drop(['level_0', 'level_1'], axis=1, inplace=True)
    testmc_df.columns = ['mc']

    print('\nCases of multicollinearity:')
    if len(testmc_df[(testmc_df.mc > 0.75) & (testmc_df.mc < 1)]) > 0:
        display(testmc_df[(testmc_df.mc > 0.75) & (testmc_df.mc < 1)])
    else:
        print('0')

In [30]:
model_evaluation(df, 'price', features_to_drop)

r^2 adjusted score: 0.6985141349587404 

Predictors where coefficient pvalue is > 0.05:
0

Predictors with Coefficient and P-value:


,coeff,pvalue
const,6.407048e+06,2.925916e-02
bedrooms,-3.983197e+04,8.281094e-89
bathrooms,4.269455e+04,1.072485e-38
sqft_living,1.054736e+02,5.972471e-09
sqft_lot,1.324820e-01,5.818407e-03
floors,7.645840e+03,3.410051e-02
waterfront,6.972981e+05,0.000000e+00
view,1.153962e+05,3.138673e-107
condition,2.704103e+04,1.612298e-30
grade,9.725809e+04,0.000000e+00



Cases of multicollinearity:


,mc
pairs,
"(sqft_living, sqft_above)",0.876448
"(sqft_above, sqft_living)",0.876448
"(grade, sqft_living)",0.762779
"(sqft_living, grade)",0.762779
"(sqft_living, sqft_living15)",0.756402
"(sqft_living15, sqft_living)",0.756402
"(grade, sqft_above)",0.756073
"(sqft_above, grade)",0.756073
"(bathrooms, sqft_living)",0.755758


[Inspiration for assumption checking](https://towardsdatascience.com/verifying-the-assumptions-of-linear-regression-in-python-and-r-f4cd2907d4c0)

In [8]:
def linearity_test(model, y):
    '''
    Function for visually inspecting the assumption of linearity in a linear regression model.
    It plots observed vs. predicted values and residuals vs. predicted values.
    
    Args:
    * model - fitted OLS model from statsmodels
    * y - observed values
    '''
    fitted_vals = model.predict()
    resids = model.resid

    fig, ax = plt.subplots(1,2)
    
    sns.regplot(x=fitted_vals, y=y, lowess=True, ax=ax[0], line_kws={'color': 'red'})
    ax[0].set_title('Observed vs. Predicted Values', fontsize=16)
    ax[0].set(xlabel='Predicted', ylabel='Observed')

    sns.regplot(x=fitted_vals, y=resids, lowess=True, ax=ax[1], line_kws={'color': 'red'})
    ax[1].set_title('Residuals vs. Predicted Values', fontsize=16)
    ax[1].set(xlabel='Predicted', ylabel='Residuals')

In [10]:
# this takes just over 2 minutes to complete
# linearity_test(model, y)